<a href="https://colab.research.google.com/github/lakshmanpadigala/Team_KLM_STS/blob/main/siamese_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install torchmetrics
!pip install pytorch_lightning
!pip install wandb
!wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 11.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from functools import partial
import re
#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel
from pytorch_lightning.loggers import WandbLogger

from sklearn.model_selection import train_test_split

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# encoder = AutoModel.from_pretrained('distilbert-base-multilingual-cased')

from transformers import BertTokenizer, BertModel

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoder = BertModel.from_pretrained('bert-base-uncased')

#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
encoder

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
EXP_NAME = 'exp_eng'
TRAIN_BATCH_SIZE = 4
DEV_BATCH_SIZE = 32
ACCUMULATE_GRAD = 8

In [ ]:
class SimilarityRegressor(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(SimilarityRegressor, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(2*self.hidden_size, self.hidden_size//2)
        self.dropout2 = nn.Dropout(p=0.2)
        self.activation2 = nn.LeakyReLU(negative_slope=0.1)
        self.linear3 = nn.Linear(self.hidden_size//2, 7)
        self.activation3 = nn.Sigmoid()

    def common_compute(self, x):
        # x = self.encoder(**x)[0][:, 0]
        x = self.encoder(**x).last_hidden_state[:, 0, :]
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        return x
    
    def forward(self, x1, x2):
        # print("Forward:",type(x1))
        # print(x1)
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        # print("Shapes after common compute:",x1.shape,x2.shape)
        x = torch.cat([torch.abs(x1 - x2), (x1 + x2)], dim=-1)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        x = 5*self.activation3(x)

        return x

In [ ]:
class LitSimilarityRegressor(pl.LightningModule):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(LitSimilarityRegressor, self).__init__()
        self.model = SimilarityRegressor(encoder, embed_size=embed_size, hidden_size=hidden_size)

        self.train_loss = torchmetrics.MeanMetric(compute_on_step=True)
        self.dev_loss = torchmetrics.MeanMetric(compute_on_step=False)
        self.test_loss = torchmetrics.MeanMetric(compute_on_step=False)

        self.train_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=True)
        self.dev_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        self.test_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        
        self.train_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=True)
        self.dev_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        self.test_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        

    def forward(self, x1, x2):
        return self.model(x1, x2)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01)

    def training_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def validation_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output[:,0], target=scores[:,0])

        return {'loss': loss, 'preds': output, 'target': scores}
    
    def test_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output[:,0], target=scores[:,0])

        return {'loss': loss, 'preds': output, 'target': scores}

    def predict_step(self, batch, batch_idx):
        x1, x2, _ = batch
        output = self(x1, x2)

        return {'preds': output[:,0]}

    def training_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.log('train/step/loss', self.train_loss(loss))
        self.log('train/step/mape', self.train_mape(preds, target))
        self.log('train/step/pcc', self.train_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,))))

    def validation_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.dev_loss(loss)
        self.dev_mape(preds[:,0], target[:,0])
        self.dev_pcc(torch.reshape(preds[:,0], (-1,)), torch.reshape(target[:,0], (-1,)))

    def test_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.test_loss(loss)
        self.test_mape(preds[:,0], target[:,0])
        self.test_pcc(torch.reshape(preds[:,0], (-1,)), torch.reshape(target[:,0], (-1,)))

    # def on_epoch_end(self, outs):
    #     self.log('train/epoch/loss', self.train_loss)
    #     self.log('train/epoch/mape', self.train_mape)
    #     self.log('train/epoch/pcc', self.train_pcc)
    
    def on_train_epoch_end(self):
        self.log('train/epoch/loss', self.train_loss)
        self.log('train/epoch/mape', self.train_mape)
        self.log('train/epoch/pcc', self.train_pcc)

    def on_validation_epoch_end(self):
        self.log('dev/loss', self.dev_loss)
        self.log('dev/mape', self.dev_mape)
        self.log('dev/pcc', self.dev_pcc)
    
    def on_test_epoch_end(self):
        self.log('dev/loss', self.test_loss)
        self.log('dev/mape', self.test_mape)
        self.log('dev/pcc', self.test_pcc)

In [ ]:
model = LitSimilarityRegressor(encoder, embed_size=768, hidden_size=256)
model

LitSimilarityRegressor(
  (model): SimilarityRegressor(
    (encoder): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768

In [ ]:
train_text = list()
# train_file = open('/content/drive/MyDrive/Colab Notebooks/Working/es_final/es_train_y.txt','r')
train_file = open('/content/drive/MyDrive/Colab Notebooks/Working/en/en-train.txt','r')
train_text_lines = train_file.readlines()
for line in train_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s2).lower()
    score = float(score.strip())
    train_text.append([s1,s2,score])
val_text = list()
# val_file = open('/content/drive/MyDrive/Colab Notebooks/Working/es_final/es_val_y.txt','r')
val_file = open('/content/drive/MyDrive/Colab Notebooks/Working/en/en-val.txt','r')
val_text_lines = val_file.readlines()
for line in val_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s2).lower()
    score = float(score.strip())
    val_text.append([s1,s2,score])
test_text = list()
# test_file = open('/content/drive/MyDrive/Colab Notebooks/Working/es_final/es_test_y.txt','r')
test_file = open('/content/drive/MyDrive/Colab Notebooks/Working/en/test-sentences-2017.txt','r')
test_text_lines = test_file.readlines()
for line in test_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', s2).lower()
    score = float(score.strip())
    test_text.append([s1,s2,score])

In [ ]:
def collate_fn(batch, tokenizer):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:
        # print("Sample from here:",sample)
        text1 = str(sample[0]).strip()
        text2 = str(sample[1]).strip()
        
        score = torch.tensor([sample[2]])

        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score.unsqueeze(0))

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
    scores = torch.cat(scores)

    return texts_1, texts_2, scores

In [ ]:
collate_partial = partial(collate_fn, tokenizer=tokenizer)
dataloader = DataLoader(train_text, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_partial)

In [ ]:
# next(iter(dataloader))

In [ ]:
class MLNSDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, dev_dataset, test_dataset, train_batch_size, dev_batch_size, collate_fn, tokenizer):
        super(MLNSDataModule, self).__init__()
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.train_batch_size = train_batch_size
        self.dev_batch_size = dev_batch_size
        self.collate_fn = collate_fn
        self.tokenizer = tokenizer

    def train_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch_size, collate_fn=collate_partial)

    def val_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.dev_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def test_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def predict_dataloader(self):
        #return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)
        return self.test_dataloader()

In [ ]:
data_module = MLNSDataModule(train_text, val_text, test_text, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
# data_module

**Training**

In [ ]:
logger = WandbLogger(save_dir=EXP_NAME, project=EXP_NAME, log_model=False)
logger

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='/content/drive/MyDrive/Colab Notebooks/Working',
    filename='en{epoch}-{step}',
    monitor='dev/pcc',
    mode='max',
    save_top_k=1,
    verbose=True,
    save_last=False,
    save_weights_only=False,
    every_n_epochs=1
)

In [ ]:
trainer = pl.Trainer(max_epochs=10,
                     accumulate_grad_batches=ACCUMULATE_GRAD,
                     accelerator='gpu',
                     check_val_every_n_epoch=1, val_check_interval=0.25,
                     enable_progress_bar=True,
                     gradient_clip_val=0.25,
                     enable_checkpointing=True,
                     logger=logger,
                     enable_model_summary=True)

# trainer
# ,callbacks=[checkpoint_callback],track_grad_norm=2,

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule=data_module)
#trainer.fit(model, datamodule=data_module, ckpt_path=root_path+'model/epoch=3-step=571.ckpt')

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                        | Params
-----------------------------------------------------------
0 | model      | SimilarityRegressor         | 109 M 
1 | train_loss | MeanMetric                  | 0     
2 | dev_loss   | MeanMetric                  | 0     
3 | test_loss  | MeanMetric                  | 0     
4 | train_mape | MeanAbsolutePercentageError | 0     
5 | dev_mape   | MeanAbsolutePercentageError | 0     
6 | test_mape  | MeanAbsolutePercentageError | 0     
7 | train_pcc  | PearsonCorrCoef             | 0     
8 | dev_pcc    | PearsonCorrCoef             | 0     
9 | test_pcc   | PearsonCorrCoef             | 0     
-----------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.983   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MeanAbsolutePercentageError was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric PearsonCorrCoef was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

<ipython-input-103-9ea8d56b1b21>:28: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(input=output, target=scores)


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
test_pred = trainer.predict(datamodule=data_module)

all_outputs = list()
for batch_outputs in test_pred:
    all_outputs.append(batch_outputs['preds'])
all_outputs = torch.cat(all_outputs, dim=0)

all_outputs.shape


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at exp_eng/exp_eng/x3d8d54y/checkpoints/epoch=9-step=4179.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at exp_eng/exp_eng/x3d8d54y/checkpoints/epoch=9-step=4179.ckpt


Predicting: 0it [00:00, ?it/s]

torch.Size([250])

In [ ]:
test_result = trainer.test(datamodule=data_module)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at exp_eng/exp_eng/x3d8d54y/checkpoints/epoch=9-step=4179.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at exp_eng/exp_eng/x3d8d54y/checkpoints/epoch=9-step=4179.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         dev/loss          │            nan            │
│         dev/mape          │            nan            │
│          dev/pcc          │            nan            │
└───────────────────────────┴───────────────────────────┘

In [ ]:
actual_outputs = [sublist[2] for sublist in test_text]

# Convert the list of second values to a tensor
actual_outputs = torch.tensor(actual_outputs)


In [ ]:
from scipy.stats import pearsonr
pcc, p_value = pearsonr(actual_outputs,all_outputs )

# Print the PCC
print("PCC: {:.5f}".format(pcc))

PCC: 0.16825


In [ ]:
actual_outputs,all_outputs

(tensor([4.2000, 1.2000, 1.0000, 1.0000, 1.2000, 2.0000, 4.6000, 0.6000, 2.8000,
         2.0000, 2.0000, 1.8000, 4.8000, 2.4000, 3.4000, 2.4000, 1.8000, 3.8000,
         1.8000, 1.6000, 5.0000, 0.0000, 4.8000, 5.0000, 0.0000, 0.4000, 5.0000,
         2.8000, 0.0000, 5.0000, 2.6000, 0.0000, 1.4000, 3.4000, 1.0000, 0.2000,
         4.8000, 2.0000, 1.4000, 3.4000, 0.0000, 3.0000, 1.6000, 3.6000, 3.8000,
         1.6000, 1.4000, 3.4000, 2.6000, 3.0000, 2.8000, 1.6000, 2.6000, 4.4000,
         0.0000, 1.6000, 0.8000, 0.0000, 5.0000, 3.0000, 2.8000, 3.4000, 2.6000,
         4.4000, 4.4000, 0.0000, 1.4000, 1.6000, 1.2000, 0.6000, 4.6000, 0.4000,
         3.8000, 1.4000, 1.4000, 2.0000, 4.4000, 5.0000, 2.6000, 0.0000, 0.0000,
         5.0000, 5.0000, 0.0000, 3.6000, 5.0000, 4.0000, 2.6000, 0.8000, 2.0000,
         1.6000, 2.2000, 1.0000, 0.0000, 1.4000, 1.6000, 1.2000, 0.6000, 0.6000,
         0.6000, 3.0000, 5.0000, 3.0000, 0.8000, 0.6000, 0.0000, 3.6000, 0.6000,
         3.4000, 2.8000, 2.6